# Контрольная работа №1 (автоматическая загрузка данных с GitHub)

Каждая задача решается в отдельном ноутбуке, код содержит комментарии, объясняющие вычисления.

---

**Источники данных:**  
- [jena_climate_2009_2016.csv](https://raw.githubusercontent.com/raffeekk/KP-TOBD1/refs/heads/main/jena_climate_2009_2016.csv)  
- [diabetes_dataset.csv](https://raw.githubusercontent.com/raffeekk/KP-TOBD1/refs/heads/main/diabetes_dataset%20(1).csv)  
- [image1.jpg](https://raw.githubusercontent.com/raffeekk/KP-TOBD1/refs/heads/main/image1%20(1).jpeg)

---

**Содержание ноутбука**:
1. Преобразование Фурье для климатических данных.
2. PCA-анализ для диабетического датасета.
3. Сжатие изображения через FFT и SVD.


In [ ]:

# Часть 1 — Fourier-преобразование для jena_climate_2009_2016.csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq
from scipy.signal import welch
import requests

# Скачиваем файл напрямую с GitHub
jena_url = "https://raw.githubusercontent.com/raffeekk/KP-TOBD1/refs/heads/main/jena_climate_2009_2016.csv"
jena_fname = "jena_climate_2009_2016.csv"
with open(jena_fname, "wb") as f:
    f.write(requests.get(jena_url).content)

df = pd.read_csv(jena_fname)
print("Колонки:", df.columns.tolist())

# Определяем колонку температуры
temp_col = [c for c in df.columns if "T" in c][0]
signal = df[temp_col].astype(float).values
n = min(len(signal), 20000)
signal = signal[:n]
signal_centered = signal - np.mean(signal)

# FFT
N = len(signal_centered)
yf = fft(signal_centered)
xf = fftfreq(N, d=1.0)
power = np.abs(yf)**2 / N

# PSD
f_welch, Pxx = welch(signal_centered, nperseg=1024)

plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(f_welch[:len(f_welch)//10], Pxx[:len(Pxx)//10])
plt.title("PSD (Welch) — низкочастотная часть")
plt.subplot(1,2,2)
plt.semilogy(np.fft.fftshift(fftfreq(N)), np.fft.fftshift(power))
plt.title("Полный спектр мощности (лог шкала)")
plt.tight_layout()
plt.show()

# Фильтрация низких частот
k_low = int(0.01 * N)
mask = np.zeros(N, dtype=bool)
mask[:k_low] = True
mask[-k_low:] = True
yf_filtered = yf * mask
reconstructed = np.real(ifft(yf_filtered)) + np.mean(signal)

plt.figure(figsize=(12,5))
plt.plot(signal, label="Оригинал")
plt.plot(reconstructed, label=f"Реконструкция (k_low={k_low})")
plt.legend()
plt.title("Временной ряд: до и после фильтрации FFT")
plt.tight_layout()
plt.show()

print("Доля мощности, сохранённой низкими частотами:", (np.abs(yf_filtered)**2/N).sum()/power.sum())


In [ ]:

# Часть 2 — PCA для diabetes_dataset.csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import requests

# Скачиваем файл напрямую с GitHub
diabetes_url = "https://raw.githubusercontent.com/raffeekk/KP-TOBD1/refs/heads/main/diabetes_dataset%20(1).csv"
fname = "diabetes_dataset.csv"
with open(fname, "wb") as f:
    f.write(requests.get(diabetes_url).content)

df = pd.read_csv(fname)
print("Размерность:", df.shape)
display(df.head())

target_col = [c for c in df.columns if "diagnosed" in c.lower()][0]
X = df.drop(columns=[target_col])
y = df[target_col]

# Кодирование категориальных
for c in X.columns:
    if X[c].dtype == "object":
        X[c] = LabelEncoder().fit_transform(X[c].astype(str))

X = X.fillna(X.mean())

Xs = StandardScaler().fit_transform(X)
pcs = PCA(n_components=2).fit_transform(Xs)

plt.figure(figsize=(8,6))
y_num = LabelEncoder().fit_transform(y.astype(str))
plt.scatter(pcs[:,0], pcs[:,1], c=y_num, cmap='coolwarm', s=20)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA — первые 2 главные компоненты (диабет/без диабета)")
plt.colorbar(label="diagnosed_diabetes")
plt.grid(True)
plt.show()


In [ ]:

# Часть 3 — Сжатие изображения через FFT и SVD
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from numpy.fft import fft2, ifft2
from scipy.linalg import svd
import requests
from io import BytesIO
import time

# Скачиваем изображение напрямую с GitHub
img_url = "https://raw.githubusercontent.com/raffeekk/KP-TOBD1/refs/heads/main/image1%20(1).jpeg"
img_fname = "image1.jpg"
with open(img_fname, "wb") as f:
    f.write(requests.get(img_url).content)

img = Image.open(img_fname).convert("RGB")
arr = np.array(img).astype(float)/255.0
h,w,c = arr.shape
print(f"Размер изображения: {w}x{h}, каналы: {c}")

def fft_compress(channel, keep_fraction=0.1):
    F = fft2(channel)
    magnitude = np.abs(F)
    threshold = np.sort(magnitude.flatten())[-int(len(magnitude.flatten())*keep_fraction)]
    mask = magnitude >= threshold
    F_comp = F * mask
    rec = np.real(ifft2(F_comp))
    return rec, mask

def svd_compress(channel, k):
    U, s, Vt = svd(channel, full_matrices=False)
    rec = (U[:, :k] * s[:k]) @ Vt[:k, :]
    return rec, s

# FFT и SVD примеры
rec_fft, _ = fft_compress(arr[:,:,0], keep_fraction=0.05)
rec_svd, s = svd_compress(arr[:,:,0], k=50)

plt.figure(figsize=(12,5))
plt.subplot(1,3,1); plt.imshow(arr); plt.title("Оригинал"); plt.axis("off")
plt.subplot(1,3,2); plt.imshow(np.clip(rec_fft,0,1), cmap="gray"); plt.title("FFT 5%"); plt.axis("off")
plt.subplot(1,3,3); plt.imshow(np.clip(rec_svd,0,1), cmap="gray"); plt.title("SVD k=50"); plt.axis("off")
plt.show()

# Сингулярные значения
plt.figure(figsize=(6,4))
plt.plot(s, marker='o')
plt.title("Сингулярные значения (канал R)")
plt.xlabel("i")
plt.ylabel("s_i")
plt.grid(True)
plt.show()
